<a href="https://colab.research.google.com/github/HR0262/AI-in-transportation-assignments/blob/main/project/projectcode_addportal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **all data extraction**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

train_df = pd.read_csv("training_dataset.csv",sep=";")
train_df

,DP_ID,PORTAL,Date,Time,Interval_1,Interval_5,Interval_15,Interval_30,Interval_60,SPEED_MS_AVG,FLOW
0,543,"E4S 56,780",20210601,04:00:00,240,48,16,8,4,19.34,10.0
1,539,"E4S 56,160",20210601,04:00:00,240,48,16,8,4,19.64,9.0
2,534,"E4S 56,780",20210601,04:00:00,240,48,16,8,4,23.36,5.0
3,351,"E4S 56,490",20210601,04:00:00,240,48,16,8,4,20.14,7.0
4,353,"E4S 57,055",20210601,04:00:00,240,48,16,8,4,17.70,7.0
...,...,...,...,...,...,...,...,...,...,...,...
1921962,353,"E4S 57,055",20211231,10:00:00,600,120,40,20,10,17.45,11.0
1921963,543,"E4S 56,780",20211231,10:00:00,600,120,40,20,10,18.09,11.0
1921964,153,"E4S 56,490",20211231,10:00:00,600,120,40,20,10,23.53,7.0
1921965,351,"E4S 56,490",20211231,10:00:00,600,120,40,20,10,18.49,9.0


In [2]:
# new dataframe for sum speed and flow
portal_avg_df = train_df.groupby(['PORTAL', 'Date', 'Time'], as_index=False).agg({
    'Interval_1': 'first',  # Keep the first value of the interval column
    'Interval_5': 'first',
    'Interval_15': 'first',
    'Interval_30': 'first',
    'Interval_60': 'first',
    'SPEED_MS_AVG': 'mean',  # Calculate average speed
    'FLOW': 'sum'  # Calculate sum flow
})

# sorting the value to original
portal_avg_df = portal_avg_df.sort_values(by=['Date', 'Time'])

# Save to CSV
# portal_avg_df.to_csv('portal_avg_speed_flow_with_intervals.csv', index=False)

# show data
portal_avg_df

,PORTAL,Date,Time,Interval_1,Interval_5,Interval_15,Interval_30,Interval_60,SPEED_MS_AVG,FLOW
0,"E4S 55,620",20210601,04:00:00,240,48,16,8,4,22.110000,29.0
74333,"E4S 56,160",20210601,04:00:00,240,48,16,8,4,23.326667,28.0
148548,"E4S 56,490",20210601,04:00:00,240,48,16,8,4,21.460000,24.0
224995,"E4S 56,780",20210601,04:00:00,240,48,16,8,4,20.047500,27.0
301341,"E4S 57,055",20210601,04:00:00,240,48,16,8,4,20.520000,27.0
...,...,...,...,...,...,...,...,...,...,...
301340,"E4S 56,780",20211231,10:00:00,600,120,40,20,10,17.907500,40.0
375372,"E4S 57,055",20211231,10:00:00,600,120,40,20,10,18.993333,42.0
451461,"E4S 57,435",20211231,10:00:00,600,120,40,20,10,21.206667,25.0
527686,"E4S 57,820",20211231,10:00:00,600,120,40,20,10,21.106667,20.0


In [3]:
# Get the unique portal values
portal_ids = portal_avg_df['PORTAL'].unique()

# Create a dictionary to hold the split dataframes
portal_avg_dfs = {}

# Loop through each PORTAL and create a separate dataframe
for portal_id in portal_ids:
    portal_avg_dfs[f'portal_avg_df_{portal_id}'] = portal_avg_df[portal_avg_df['PORTAL'] == portal_id]
print(portal_ids)

['E4S 55,620' 'E4S 56,160' 'E4S 56,490' 'E4S 56,780' 'E4S 57,055'
 'E4S 57,435' 'E4S 57,820' 'E4S 58,140']


In [4]:
from datetime import datetime
# 创建一个空的列表来存储每个portal的向量化数据集  Create an empty list to store the vectorised dataset for each portal
vectorized_portal_datasets = []
nintvals=361
# 首先，找到所有portal中出现的唯一的日期  First, find the unique date that appears in all portals
all_days = set()
for portal in portal_ids:
    df = portal_avg_dfs[f'portal_avg_df_{portal}']
    df_sorted = df.sort_values(["Date", "Interval_1"])
    all_days.update(df_sorted['Date'].unique())
all_days = sorted(all_days)
all_days_formatted = [datetime.strptime(str(day), '%Y%m%d').date() for day in all_days]
print(all_days)

# 循环遍历每个portal  Loop through each portal
for portal in portal_ids:
    df = portal_avg_dfs[f'portal_avg_df_{portal}']
    df_sorted = df.sort_values(["Date", "Interval_1"])

    # 提取唯一的日期  Extract unique date
    days = np.unique(df_sorted['Date'].values)

    # 计算总天数  Calculate the total number of days
    ndays = len(days)

    # 按日期分组  Grouping by date
    day_subsets_df = df_sorted.groupby(["Date"])

    # 创建一个填满NaN值的矩阵  Create a matrix filled with NaN values
    vectorized_day_dataset = np.full((len(all_days), nintvals), np.nan)

    # 循环遍历每个唯一的天  Loop through each unique day
    for i, day in enumerate(all_days):
        if day in days:
            df_t = day_subsets_df.get_group(day)

            # 循环遍历当前天的DataFrame的每一行  Loop through each row of the DataFrame for the current day
            for j in range(len(df_t)):
                # 提取"Interval_1"和"FLOW"值，并填充'vectorized_day_dataset'  Extract the 'Interval_1' and 'FLOW' values and populate the 'vectorized_day_dataset'
                interval_index = df_t.iloc[j]["Interval_1"] - 240
                if 0 <= interval_index < nintvals:  # 确保索引有效  Make sure the index is valid
                    vectorized_day_dataset[i, interval_index] = df_t.iloc[j]["FLOW"]

    # 将当前portal的向量化数据集添加到列表中  Add the vectorised dataset of the current portal to the list
    vectorized_portal_datasets.append(vectorized_day_dataset)

# 将所有向量化数据集合并为一个大的三维数组  Merge all vectorised datasets into one large 3D array
final_vectorized_dataset = np.array(vectorized_portal_datasets)

# 打印结果  Print the results
print(final_vectorized_dataset)
print(final_vectorized_dataset.shape)

[20210601, 20210602, 20210603, 20210604, 20210605, 20210606, 20210607, 20210608, 20210609, 20210610, 20210611, 20210612, 20210613, 20210614, 20210615, 20210616, 20210617, 20210618, 20210619, 20210620, 20210621, 20210622, 20210623, 20210624, 20210625, 20210626, 20210627, 20210628, 20210629, 20210630, 20210701, 20210702, 20210703, 20210704, 20210705, 20210706, 20210707, 20210708, 20210709, 20210710, 20210711, 20210712, 20210713, 20210714, 20210715, 20210716, 20210717, 20210718, 20210719, 20210720, 20210721, 20210722, 20210723, 20210724, 20210725, 20210726, 20210727, 20210728, 20210729, 20210730, 20210731, 20210801, 20210802, 20210803, 20210804, 20210805, 20210806, 20210807, 20210808, 20210809, 20210810, 20210811, 20210812, 20210813, 20210814, 20210815, 20210816, 20210817, 20210818, 20210819, 20210820, 20210821, 20210822, 20210823, 20210824, 20210825, 20210826, 20210827, 20210828, 20210829, 20210830, 20210831, 20210901, 20210902, 20210903, 20210904, 20210905, 20210906, 20210907, 20210908,

<ipython-input-4-ff6e5d0f9f36>:35: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  df_t = day_subsets_df.get_group(day)


[[[29. 26. 38. ... 60. 46. 68.]
  [29. 23. 29. ... 54. 59. 59.]
  [37. 35. 36. ... 57. 40. 53.]
  ...
  [ 1.  6.  3. ... 50. 47. 45.]
  [ 6.  3.  6. ... 42. 46. 62.]
  [ 2.  3.  1. ... 24. 14. 16.]]

 [[28. 33. 42. ... 47. 59. 61.]
  [26. 29. 35. ... 54. 59. 52.]
  [34. 38. 27. ... 45. 49. 51.]
  ...
  [ 3.  4.  4. ... 50. 43. 56.]
  [ 4.  5.  6. ... 38. 54. 57.]
  [ 2.  1.  1. ... 14. 16. 27.]]

 [[24. 38. 43. ... 55. 64. 69.]
  [25. 35. 37. ... 60. 64. 57.]
  [35. 36. 34. ... 50. 55. 58.]
  ...
  [ 4.  6.  2. ... 49. 53. 62.]
  [ 3.  7.  7. ... 48. 58. 56.]
  [ 2.  3.  2. ... 18. 13. 42.]]

 ...

 [[ 4.  3.  5. ... 58. 54. 69.]
  [ 3.  2.  1. ... 58. 57. 66.]
  [ 8.  4.  6. ... 59. 61. 66.]
  ...
  [ 4.  1.  6. ... 49. 39. 44.]
  [ 2.  4.  8. ... 36. 40. 28.]
  [ 1.  3.  4. ... 22. 23. 25.]]

 [[ 4.  3.  7. ... 50. 60. 72.]
  [ 3.  1.  4. ... 60. 57. 60.]
  [ 8.  3.  7. ... 59. 52. 68.]
  ...
  [ 2.  3.  5. ... 48. 49. 30.]
  [ 1.  7.  7. ... 31. 43. 33.]
  [nan  4.  4. ... 18. 24. 2

In [5]:
#提取speed  Extraction speed
vectorized_portal_datasets=[]
for portal in portal_ids:

    df = portal_avg_dfs[f'portal_avg_df_{portal}']
    df_sorted = df.sort_values(["Date", "Interval_1"])

    # 提取唯一的日期  Extract unique date
    days = np.unique(df_sorted['Date'].values)

    # 计算总天数  Calculate the total number of days
    ndays = len(days)

    # 按日期分组  Grouping by date
    day_subsets_df = df_sorted.groupby(["Date"])

    # 创建一个填满NaN值的矩阵  Create a matrix filled with NaN values
    vectorized_day_dataset = np.full((len(all_days), nintvals), np.nan)

    # 循环遍历每个唯一的天  Loop through each unique day
    for i, day in enumerate(all_days):
        if day in days:
            df_t = day_subsets_df.get_group(day)

            # 循环遍历当前天的DataFrame的每一行  Loop through each row of the DataFrame for the current day
            for j in range(len(df_t)):
                # 提取"Interval_1"和"FLOW"值，并填充'vectorized_day_dataset'  Extract the 'Interval_1' and 'FLOW' values and populate the 'vectorized_day_dataset'
                interval_index = df_t.iloc[j]["Interval_1"] - 240
                if 0 <= interval_index < nintvals:  # 确保索引有效  Make sure the index is valid
                    vectorized_day_dataset[i, interval_index] = df_t.iloc[j]["SPEED_MS_AVG"]
    # 将当前portal的向量化数据集添加到列表中  Add the vectorised dataset of the current portal to the list
    vectorized_portal_datasets.append(vectorized_day_dataset)

# 将所有向量化数据集合并为一个大的三维数组  Merge all vectorised datasets into one large 3D array
final_vectorized_dataset_speed = np.array(vectorized_portal_datasets)

# 打印结果  Print the results
print(final_vectorized_dataset_speed)
print(final_vectorized_dataset_speed.shape)

<ipython-input-5-a8515002362a>:23: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  df_t = day_subsets_df.get_group(day)


[[[22.11       23.53333333 21.83       ... 19.09       17.96
   19.48333333]
  [23.61333333 22.45666667 22.78333333 ... 19.40333333 19.56666667
   19.52666667]
  [23.56666667 22.20666667 23.28       ... 19.89333333 19.85
   20.33666667]
  ...
  [18.6        19.77       19.9        ... 18.89       19.54333333
   19.15333333]
  [23.05       22.23       21.415      ... 20.14333333 19.26666667
   18.53333333]
  [21.4        25.35       20.3        ... 20.84       21.23666667
   22.44      ]]

 [[23.32666667 24.41       22.94666667 ... 20.08666667 20.12333333
   20.01333333]
  [25.17333333 23.93333333 23.44       ... 21.16666667 20.85666667
   20.42      ]
  [23.59       22.71333333 24.62       ... 21.77       20.76
   20.39666667]
  ...
  [18.125      20.95       20.865      ... 20.23333333 20.26666667
   19.82      ]
  [22.515      24.7        21.9        ... 21.01333333 20.04333333
   20.28333333]
  [25.7        26.1        20.8        ... 21.81       22.36666667
   21.29666667]]

 [[21.

In [6]:
#插值  interpolation

# 自定义函数进行周期性插值  Custom functions for periodic interpolation
def periodic_interpolation(df, period):
    df = pd.DataFrame(df)
    # 对于每一列（代表一个时间间隔）  For each column (representing a time interval)
    for col in df.columns:
        # 遍历每一行  Go through each row
        for i in range(len(df)):
            # 如果当前行有缺失值  If the current row has a missing value
            if np.isnan(df[col][i]):
                # 向前找到当前周期中对应的行（例如，如果今天是星期三，我们查看上一个星期三）  Move forward to find the corresponding row in the current period (e.g. if today is Wednesday, we look at the previous Wednesday)
                idx_left = (i + period) % len(df)
                # 如果对应的行也有缺失值，继续向后查找直到找到非缺失值  If the corresponding row also has a missing value, continue looking backwards until you find a non-missing value
                while np.isnan(df[col][idx_left]) and idx_left>=0:
                    idx_left = (idx_left + period) % len(df)
                # 向后找  Look backwards
                idx_right = (i + period) % len(df)
                while np.isnan(df[col][idx_right]) and idx_right<len(df):
                    idx_right = (idx_right + period) % len(df)
                # 用找到的非缺失值线性填充当前缺失值  The currently missing value is linearly populated with the found non-missing value
                if idx_right>=len(df):
                  df[col][i] = df[col][idx_left]
                elif idx_left<=0:
                  df[col][i] = df[col][idx_right]
                else:
                  df[col][i] = np.mean([df[col][idx_left], df[col][idx_right]])
    df = df.to_numpy()
    return df

def periodic_interpolation_3d(df, period):
    df_interpolated = np.zeros(df.shape)
    time_steps, period_length, num_features = df.shape
    for t in range(time_steps):
      df_interpolated[t] = periodic_interpolation(df[t], period)

    return df_interpolated

# # 应用周期性插值  Apply periodic interpolation
# df_interpolated = periodic_interpolation(df, 7*8)


# # 将插值后的 DataFrame 转换回 NumPy 数组  Convert the interpolated DataFrame back to a NumPy array
# final_vectorized_dataset_2d = df_interpolated.to_numpy()

# print(final_vectorized_dataset_2d)
# print(final_vectorized_dataset_2d.shape)

In [7]:
final_vectorized_dataset_speed_interpolated = periodic_interpolation_3d(final_vectorized_dataset_speed, 7)
final_vectorized_dataset_interpolated = periodic_interpolation_3d(final_vectorized_dataset, 7)
print(final_vectorized_dataset_speed_interpolated.shape)
print(final_vectorized_dataset_interpolated.shape)


<ipython-input-6-3cb84ca276b7>:27: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df[col][i] = np.mean([df[col][idx_left], df[col][idx_right]])
<ipython-input-6-3cb84ca276b7>:25: FutureWarning: ChainedAssignmentError: behaviour will change in

(8, 214, 361)
(8, 214, 361)


In [8]:
np.save('final_vectorized_dataset_speed_interpolated.npy', final_vectorized_dataset_speed_interpolated)
np.save('final_vectorized_dataset_interpolated.npy', final_vectorized_dataset_interpolated)

# **change portal current here**


In [9]:
final_vectorized_dataset_speed_interpolated = np.load('final_vectorized_dataset_speed_interpolated.npy')
final_vectorized_dataset_interpolated = np.load('final_vectorized_dataset_interpolated.npy')

In [40]:
portal_current = 2

Time series for forecasting----xgboost prediction

mashine learning--30speed, 30flow, 2labels
lstm--30speed, 30flow


Need to split the data into x-62 values, y-next minute values

Remember to divide into training set validation set



In [60]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder

def datadivide(arr_speed, arr_flow, portal_current, lag_size = 30, weekend = False, startday = 0, time = 0):
  print(arr_speed[portal_current].shape)
  rows, columns = arr_speed[portal_current].shape

  x = []
  y = []

  # encoder = OneHotEncoder()
  # one_hot_labels = encoder.fit_transform(cluster_labels.reshape(-1, 1))
  # one_hot_labels = one_hot_labels.toarray()

  # 遍历数组，除了最后 lag_size 个元素  Iterate through the array, except for the last lag_size elements
  for i in range(1, rows):
    for j in range(columns - lag_size):
        # x_features = (
        #         list(arr[portal_current-2][i][j+15:j+lag_size].flatten()) +
        #         list(arr[portal_current-1][i][j+15:j+lag_size].flatten()) +
        #         list(arr[portal_current][i][j:j+lag_size].flatten()) +
        #         list(arr[portal_current][i-7][j+15:j+lag_size].flatten())
        #     )

        x_features = (list(arr_speed[portal_current][i][j:j+lag_size].flatten()) + list(arr_flow[portal_current][i][j:j+lag_size].flatten()))
        # x_features = np.append(x_features, np.mean(arr[portal_current-2][i][j+15:j+lag_size].flatten()))
        x_features = np.append(x_features, np.mean(arr_flow[portal_current-1][i][j+15:j+lag_size].flatten()))
        x_features = np.append(x_features, np.mean(arr_flow[portal_current][i-7][j+15:j+lag_size].flatten()))
        # y 是当前时间步长的数据  y is the data for the current time step
        y_value = np.mean(arr_flow[portal_current][i][j+lag_size:j+lag_size+15])
        # 加入当前时间  Add the current time
        if time:
          x_features = np.append(x_features, j//time)
        if weekend:
          if i%7+startday ==6 or i%7+startday ==7:
            x_features = np.append(x_features, 1)
          else:
            x_features = np.append(x_features, 0)


        # 将它们添加到相应的列表中  Add them to the appropriate list
        x.append(x_features)
        y.append(y_value)

  # translate it to NumPy array
  x = np.array(x)
  y = np.array(y)

  return x, y

In [61]:
x, y = datadivide(final_vectorized_dataset_speed_interpolated, final_vectorized_dataset_interpolated, portal_current, time=5, weekend = True, startday = 2)
print(x.shape)
print(y.shape)

(214, 361)
(70503, 64)
(70503,)


pca

In [62]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()  #standardize it
x_scaled = scaler.fit_transform(x)


# 分离出需要进行PCA降维的特征和不需要降维的特征  Separate the features that require PCA dimensionality reduction and those that do not
features_to_reduce = x[:, :60]  #speed and flow
features_to_keep = x[:, 60:]    #cluster labels and time

pca = PCA(n_components=10)  # Adjust the number of components (dimensions) as needed
features_reduced = pca.fit_transform(features_to_reduce)
print(features_reduced.shape)

explained_variance = pca.explained_variance_ratio_
print(f'Explained Variance Ratio: {explained_variance}')
print(f"Sum of the explained variance: {sum(explained_variance)}")

# 将降维后的特征和原始特征拼接起来  Stitch together the reduced features and the original features
data_transformed = np.hstack((features_reduced, features_to_keep))
print(data_transformed.shape)

# data_transformed = x_scaled

(70503, 10)
Explained Variance Ratio: [0.83704045 0.01991762 0.00859891 0.0062334  0.0057458  0.00507469
 0.00487718 0.00482965 0.00482483 0.00480035]
Sum of the explained variance: 0.9019428818701248
(70503, 14)


tran test split ----test size 0.2

Can't be divided randomly because the output will be contained within the inputs

In [63]:
test_size = 0.2

x_train_pca = data_transformed[:int(len(x)*(1-test_size))]
x_test_pca = data_transformed[int(len(x)*(1-test_size)):]
y_train = y[:int(len(y)*(1-test_size))]
y_test = y[int(len(y)*(1-test_size)):]

print(x_train_pca.shape)
print(x_test_pca.shape)
print(y_train.shape)
print(y_test.shape)


(56402, 14)
(14101, 14)
(56402,)
(14101,)


In [25]:
pip install catboost
from mmap import MAP_EXECUTABLE
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor

In [64]:
# # 定义参数分布  Define the parameter distribution
# param_distribution = {
#     'n_estimators': randint(100, 500),
#     'learning_rate': uniform(0.01, 0.2),
#     'max_depth': randint(3, 10),
#     'min_child_weight': randint(1, 10),
# }

# # 创建XGBRegressor模型  Create an XGBRegressor model
# xgb_reg = XGBRegressor()

# # 创建RandomizedSearchCV对象  Create a RandomizedSearchCV object
# random_search = RandomizedSearchCV(
#     estimator=xgb_reg,
#     param_distributions=param_distribution,
#     n_iter=100,  # 可以根据需要调整迭代次数  You can adjust the number of iterations as needed
#     scoring='neg_mean_squared_error',
#     cv=3,
#     random_state=42
# )

# # 执行随机搜索  Perform a random search
# random_search.fit(x_train_pca, y_train)

# # 输出最佳参数  Output the best parameters
# print("Best parameters found: ", random_search.best_params_)

# # 输出最佳模型得分  Output the best model score
# print("Best model score: ", random_search.best_score_)

# different models
# model = LinearRegression()
# model = XGBRegressor()
# model = RandomForestRegressor()
# model = SVR()
# model = DecisionTreeRegressor()
model = CatBoostRegressor(verbose=0)
# model = lgb.LGBMRegressor()
# model = GradientBoostingRegressor()
# model = AdaBoostRegressor()



model.fit(x_train_pca, y_train)
y_pred = model.predict(x_test_pca)
print(f"Catboost model saving...")
model.save_model(f'best_catboost_model_portal_{portal_current+1}.cbm', format="cbm")

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae}")
print(f"Mean Absolute Percentage Error: {mape}")
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Catboost model saving...
Mean Absolute Error: 2.9408597599129322
Mean Absolute Percentage Error: 0.1261550446848741
Mean Squared Error: 16.90229091405198
R-squared: 0.9461554415821408


In [ ]:
#lstm
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

# 1. 数据准备 - 滑动窗口生成特征  Data Preparation - Sliding window generates features
def create_lstm_data(data, lag_size, weekend = False, startday = 0, time = 0):
    x = []
    y = []
    Weekend = []
    Time = []
    rows, columns = data.shape
    for i in range(rows):
        for j in range(columns - lag_size):
            x_features = data[i][j:j+lag_size]
            y_value = np.mean(data[i][j+lag_size:j+lag_size+15])
            # 加入当前时间  Add the current time
            if time:
              Time.append(j//time)
            # 加入是否是周末 Add weekend
            if weekend:
              if i%7+startday ==6 or i%7+startday ==7:
                Weekend.append(1)
              else:
                Weekend.append(0)

            x.append(x_features)
            y.append(y_value)
    return np.array(x), np.array(y) ,np.array(Weekend), np.array(Time)

# 假设 `vectorized_day_dataset` 是你的完整时间序列数据矩阵  Let's say 'vectorized_day_dataset' is your full time series data matrix
data1 = vectorized_day_dataset_speed0_interpolated
data2 = vectorized_day_dataset_flow0_interpolated

# 3. 生成带有聚类标签的时间序列特征数据  Generate time series feature data with clustered labels
lag_size = 30  # 滑动窗口的大小，使用前 30 个时间步预测下一个时间步  Sliding the size of the window, using the first 30 time steps to predict the next time step
x1, y1, w1, t1 = create_lstm_data(data1, lag_size, weekend = True, startday = 2, time = 5)
x2, y2, w2, t2 = create_lstm_data(data2, lag_size, weekend = True, startday = 2, time = 5)
y = y2

#PCA Dimensionality Reduction

def pca_reduction(x, n_components = 10):

  pca = PCA(n_components=n_components)  # Adjust the number of components (dimensions) as needed
  features_reduced = pca.fit_transform(x)
  print('feature reduced shape', features_reduced.shape)

  explained_variance = pca.explained_variance_ratio_
  print(f'Explained Variance Ratio: {explained_variance}')
  print(f"Sum of the explained variance: {sum(explained_variance)}")

  return features_reduced

x1 = pca_reduction(x1, n_components = 8)
x2 = pca_reduction(x2, n_components = 8)


# 4. 数据划分为训练集和测试集  The data is divided into training sets and test sets
test_size = 0.2
split_index = int(len(x) * (1 - test_size))
# x_train, x_test = x[:split_index], x[split_index:]
# y_train, y_test = y[:split_index], y[split_index:]
x1_train, x1_test = x1[:split_index], x1[split_index:]
y_train, y_test = y[:split_index], y[split_index:]
x2_train, x2_test = x2[:split_index], x2[split_index:]
w2_train, w2_test = w2[:split_index], w2[split_index:]
t2_train, t2_test = t2[:split_index], t2[split_index:]


# 5. 数据标准化  Data standardisation
scaler1 = StandardScaler()
scaler2 = StandardScaler()
scaler3 = StandardScaler()
scaler4 = StandardScaler()

x1_train_scaled = scaler1.fit_transform(x1_train.reshape(-1, x1_train.shape[-1])).reshape(x1_train.shape)
x1_test_scaled = scaler1.transform(x1_test.reshape(-1, x1_test.shape[-1])).reshape(x1_test.shape)
x2_train_scaled = scaler2.fit_transform(x2_train.reshape(-1, x2_train.shape[-1])).reshape(x2_train.shape)
x2_test_scaled = scaler2.transform(x2_test.reshape(-1, x2_test.shape[-1])).reshape(x2_test.shape)
w2_train_scaled = scaler3.fit_transform(w2_train.reshape(-1, 1)).reshape(w2_train.shape)
w2_test_scaled = scaler3.fit_transform(w2_test.reshape(-1, 1)).reshape(w2_test.shape)
t2_train_scaled = scaler4.fit_transform(t2_train.reshape(-1, 1)).reshape(t2_train.shape)
t2_test_scaled = scaler4.transform(t2_test.reshape(-1, 1)).reshape(t2_test.shape)


# x1_train_scaled = x1_train
# x2_train_scaled = x2_train

# 6. 将数据调整为适合 LSTM 的三维形式 (样本数, 时间步长 + 簇标签, 特征数)  Adjust the data to a 3D format suitable for the LSTM (number of samples, time step size + cluster labels, number of features)
# x_train_scaled = x_train_scaled.reshape((x_train_scaled.shape[0], lag_size, 2))
# x_test_scaled = x_test_scaled.reshape((x_test_scaled.shape[0], lag_size, 2))
w2_train_scaled = np.repeat(np.expand_dims(w2_train_scaled, axis=1), 8, axis=1)
w2_test_scaled = np.repeat(np.expand_dims(w2_test_scaled, axis=1), 8, axis=1)
t2_train_scaled = np.repeat(np.expand_dims(t2_train_scaled, axis=1), 8, axis=1)
t2_test_scaled = np.repeat(np.expand_dims(t2_test_scaled, axis=1), 8, axis=1)
x_train_scaled = np.stack((x1_train_scaled, x2_train_scaled, w2_train_scaled, t2_train_scaled), axis=-1)
x_test_scaled = np.stack((x1_test_scaled, x2_test_scaled, w2_test_scaled, t2_test_scaled), axis=-1)
print(x_train_scaled.shape)
print(x_test_scaled.shape)


# print(x_train_scaled)
# print(x_train_scaled.shape)
# print(y)




feature reduced shape (70834, 8)
Explained Variance Ratio: [0.51423221 0.04465434 0.02543322 0.02028904 0.0184092  0.01773303
 0.0172798  0.01676163]
Sum of the explained variance: 0.6747924668039272
feature reduced shape (70834, 8)
Explained Variance Ratio: [0.91108144 0.0131568  0.00547131 0.00335478 0.0028367  0.00281892
 0.00278137 0.00276042]
Sum of the explained variance: 0.9442617375447572
(56667, 8, 4)
(14167, 8, 4)


In [ ]:
from keras.callbacks import Callback
class MetricsCallback(Callback):
  def __init__(self, validation_data):
    super(MetricsCallback, self).__init__()
    self.validation_data = validation_data
    self.best_r2 = 0

  def on_epoch_end(self, epoch, logs=None):
    # 从初始化时传入的验证数据中获取 x_val 和 y_val  Get the x_val and y_val from the validation data passed in at initialisation
    x_val, y_val = self.validation_data

    # 使用模型进行预测  Use models to make predictions
    y_pred = self.model.predict(x_val)

    # 计算评估指标  Calculate the evaluation metrics
    mae = mean_absolute_error(y_val, y_pred)
    mse = mean_squared_error(y_val, y_pred)
    mape = mean_absolute_percentage_error(y_val, y_pred)
    r2 = r2_score(y_val, y_pred)

    # 输出评估指标  Output evaluation metrics
    print(f"\nEpoch {epoch+1} - Metrics:")
    print(f"Mean Absolute Error (MAE): {mae}")
    print(f"Mean Squared Error (MSE): {mse}")
    print(f"Mean Absolute Percentage Error (MAPE): {mape}")
    print(f"R-squared (R²): {r2}")

    if r2 > self.best_r2:
            self.best_r2 = r2
            print(f"New best model at epoch {epoch+1}, saving...")
            self.model.save(f'best_LSTM_model_portal_{portal_current+1}_epoch_{epoch+1}.keras')


In [ ]:
# 7. 创建 LSTM 模型  Create an LSTM model
from keras.layers import BatchNormalization
from keras.regularizers import l1_l2, l1, l2
model_lstm = Sequential()
model_lstm.add(LSTM(50, activation='elu', return_sequences=True, input_shape=(x_train_scaled.shape[1], 4), kernel_regularizer=l1_l2(l1=0.01, l2=0.01)))
model_lstm.add(BatchNormalization())
model_lstm.add(LSTM(50, activation='elu', return_sequences=True, kernel_regularizer=l1_l2(l1=0.01, l2=0.01)))
model_lstm.add(Dropout(0.2))
model_lstm.add(LSTM(50, activation='elu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)))
model_lstm.add(Dense(1, kernel_regularizer=l1_l2(l1=0.01, l2=0.01)))
model_lstm.compile(optimizer='adam', loss='mean_squared_error')

# 定义回调  Define callbacks
metrics_callback = MetricsCallback(validation_data=(x_test_scaled, y_test))

# 训练模型  Train the model
model_lstm.fit(x_train_scaled, y_train, epochs=25, batch_size=32, verbose=1, validation_data=(x_test_scaled, y_test), callbacks=[metrics_callback])

# 9. 预测并评估模型  Predict and evaluate models
y_pred = model_lstm.predict(x_test_scaled)

mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae}")
print(f"Mean Absolute Percentage Error: {mape}")
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Epoch 1/25


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


443/443 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step

Epoch 1 - Metrics:
Mean Absolute Error (MAE): 2.8383424695249166
Mean Squared Error (MSE): 17.128031916272427
Mean Absolute Percentage Error (MAPE): 0.12441406163418084
R-squared (R²): 0.9630741780246256
New best model at epoch 1, saving...
1771/1771 ━━━━━━━━━━━━━━━━━━━━ 48s 24ms/step - loss: 159.6626 - val_loss: 28.2406
Epoch 2/25
443/443 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step

Epoch 2 - Metrics:
Mean Absolute Error (MAE): 3.0939175947851534
Mean Squared Error (MSE): 19.136936896906263
Mean Absolute Percentage Error (MAPE): 0.14196177201906926
R-squared (R²): 0.9587432386590904
1771/1771 ━━━━━━━━━━━━━━━━━━━━ 43s 24ms/step - loss: 24.6887 - val_loss: 25.7570
Epoch 3/25
443/443 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step

Epoch 3 - Metrics:
Mean Absolute Error (MAE): 2.7173213825150153
Mean Squared Error (MSE): 15.824335590888612
Mean Absolute Percentage Error (MAPE): 0.12837749237552482
R-squared (R²): 0.9658847787203971
New best model at epoch 3, saving...
177